In [30]:
import pandas as pd
#import pandavro as pda
import lxml.html as lh
from urllib.request import urlopen, Request
import requests
pd.__version__


'1.0.1'

In [31]:
url = "http://www.brewunited.com/grain_database.php"
print(url)

http://www.brewunited.com/grain_database.php


In [32]:


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(url=url, headers=headers) 
page = requests.get(url)

print(page) 
doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
print(tr_elements)


#Check the length of the first 50 rows
[len(T) for T in tr_elements[:50]]


<Response [200]>
[<Element tr at 0x1c924210b38>, <Element tr at 0x1c924210b88>, <Element tr at 0x1c924210bd8>, <Element tr at 0x1c924210a48>, <Element tr at 0x1c92420b6d8>, <Element tr at 0x1c9241398b8>, <Element tr at 0x1c924139688>, <Element tr at 0x1c924216318>, <Element tr at 0x1c924216098>, <Element tr at 0x1c924216188>, <Element tr at 0x1c924216138>, <Element tr at 0x1c924216228>, <Element tr at 0x1c924216278>, <Element tr at 0x1c9242161d8>, <Element tr at 0x1c9242162c8>, <Element tr at 0x1c9242160e8>, <Element tr at 0x1c924213868>, <Element tr at 0x1c924213c78>, <Element tr at 0x1c9242133b8>, <Element tr at 0x1c924213318>, <Element tr at 0x1c924213b88>, <Element tr at 0x1c924213b38>, <Element tr at 0x1c924213c28>, <Element tr at 0x1c924213bd8>, <Element tr at 0x1c924268ae8>, <Element tr at 0x1c92420e908>, <Element tr at 0x1c92420e688>, <Element tr at 0x1c92420e318>, <Element tr at 0x1c92420ef48>, <Element tr at 0x1c92426ea98>, <Element tr at 0x1c92426e868>, <Element tr at 0x1c92

[8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

In [33]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Grain"
2:"Origin"
3:"Mash?"
4:"Color"
5:"Power"
6:"Potential"
7:"Max %"
8:"Notes"


In [34]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [35]:
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(20) 

,Grain,Origin,Mash?,Color,Power,Potential,Max %,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [36]:
df.count()

Grain        258
Origin       258
Mash?        258
Color        258
Power        258
Potential    258
Max %        258
Notes        258
dtype: int64

In [9]:
##cleaning Duplicates and rubish
dfnew = df.drop_duplicates()
dfnew.count()



#df = df[df['Hop'] != 'Hop']
#df = df[df['Hop'] != '0']

Grain        183
Origin       183
Mash?        183
Color        183
Power        183
Potential    183
Max %        183
Notes        183
dtype: int64

In [43]:

df = df[df['Origin'] != 'Origin']

df.count()

Grain         237
Origin        237
Mash          237
Color         237
Power         237
Potential     237
MaxPercent    237
Notes         237
dtype: int64

In [44]:
df.head(20)

,Grain,Origin,Mash,Color,Power,Potential,MaxPercent,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [45]:
df.to_json (r'C:\Users\bceolincamar\Documents\GitHub\homebrew\Export_Grains.json', orient='records')

In [40]:
df.rename(columns={ df.columns[6]: "MaxPercent" }, inplace = True)
df.rename(columns={ df.columns[2]: "Mash" }, inplace = True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Grain       258 non-null    object
 1   Origin      258 non-null    object
 2   Mash        258 non-null    object
 3   Color       258 non-null    object
 4   Power       258 non-null    object
 5   Potential   258 non-null    object
 6   MaxPercent  258 non-null    object
 7   Notes       258 non-null    object
dtypes: object(8)
memory usage: 16.2+ KB
